In [7]:
import keras as k
import keras.backend as b
import keras.layers as l
import numpy as np
import sys
import theano as t
import keras.initializations as i
sys.path.append('../src/mane/')

In [2]:
import graph as g
fb = g.graph_from_pickle('../src/mane/data/egonets.graph')

In [12]:
def row_dot(inputs):
    return b.batch_dot(inputs[0], inputs[1], axes=[1,1])
def nce_loss(y_true, y_pred):
    y_true = y_true.reshape(y_pred.shape)
    return -b.log(b.sigmoid(y_pred * y_true))
def init_uniform(shape, name=None):
    return i.uniform(shape=shape, scale=1, name=name)

In [9]:
t_in = l.Input(batch_shape=(100,1), dtype='int32', name='target')
c_in = l.Input(batch_shape=(100,1), dtype='int32', name='class')

In [20]:
emb_t = l.Embedding(input_dim=len(fb), output_dim=200, 
                    name='target_emb', input_length=1,
                    init=init_uniform) (t_in)
emb_t = l.Reshape(target_shape=(200,))(emb_t)
emb_c = l.Embedding(input_dim=len(fb), output_dim=200, 
                    name='nce_emb', input_length=1,
                    init=init_uniform) (c_in)
emb_c = l.Reshape(target_shape=(200,))(emb_c)
emb_b = l.Embedding(input_dim=len(fb), output_dim=1,
                    name='nce_bias', input_length=1,
                    init='zero') (c_in)
emb_b = l.Reshape(target_shape=(1,))(emb_b)

In [50]:
dot_prod = l.Merge(mode=row_dot, output_shape=(1,), name='row_wise_dot')([emb_t, emb_c])
logits = l.Merge(mode='sum', output_shape=(1,), name='logits')([dot_prod, emb_b])
sigm = l.Activation('sigmoid', name='label')(logits)

In [51]:
model = k.models.Model(input=[t_in, c_in], output=sigm)

In [52]:
model.compile(loss='binary_crossentropy', optimizer='adam', name='mmodel')

In [53]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
class (InputLayer)               (100, 1)              0                                            
____________________________________________________________________________________________________
target (InputLayer)              (100, 1)              0                                            
____________________________________________________________________________________________________
nce_emb (Embedding)              (100, 1, 200)         807800      class[0][0]                      
____________________________________________________________________________________________________
target_emb (Embedding)           (100, 1, 200)         807800      target[0][0]                     
___________________________________________________________________________________________

In [35]:
data_generator = fb.gen_walk('random_walk')

In [40]:
la = next(data_generator)

In [48]:
la[1]['label'] = np.array([ 1.,  1.,  1.,  1.,  1., 0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

In [58]:
model.fit(batch_size=10, x=la[0], y=la[1])

Epoch 1/10
10/10 [==============================] - 0s - loss: 0.0453
Epoch 2/10
10/10 [==============================] - 0s - loss: 0.0428
Epoch 3/10
10/10 [==============================] - 0s - loss: 0.0406
Epoch 4/10
10/10 [==============================] - 0s - loss: 0.0385
Epoch 5/10
10/10 [==============================] - 0s - loss: 0.0367
Epoch 6/10
10/10 [==============================] - 0s - loss: 0.0350
Epoch 7/10
10/10 [==============================] - 0s - loss: 0.0334
Epoch 8/10
10/10 [==============================] - 0s - loss: 0.0320
Epoch 9/10
10/10 [==============================] - 0s - loss: 0.0307
Epoch 10/10
10/10 [==============================] - 0s - loss: 0.0295
